In [0]:
# Load the dataset from DBFS (Databricks Filesystem)
file_path = "/databricks-datasets/samples/population-vs-price/data_geo.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.printSchema()
df.show(5)

root
 |-- 2014 rank: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- 2014 Population estimate: integer (nullable = true)
 |-- 2015 median sales price: double (nullable = true)

+---------+-------------+-------+----------+------------------------+-----------------------+
|2014 rank|         City|  State|State Code|2014 Population estimate|2015 median sales price|
+---------+-------------+-------+----------+------------------------+-----------------------+
|      101|   Birmingham|Alabama|        AL|                  212247|                  162.9|
|      125|   Huntsville|Alabama|        AL|                  188226|                  157.7|
|      122|       Mobile|Alabama|        AL|                  194675|                  122.5|
|      114|   Montgomery|Alabama|        AL|                  200481|                  129.0|
|       64|Anchorage[19]| Alaska|        AK|                  30101

In [0]:
# Select relevant columns for analysis
df_selected = df.select(
    "City", "State", "State Code", 
    "2014 Population estimate", "2015 median sales price"
)
df_selected.show(5)

+-------------+-------+----------+------------------------+-----------------------+
|         City|  State|State Code|2014 Population estimate|2015 median sales price|
+-------------+-------+----------+------------------------+-----------------------+
|   Birmingham|Alabama|        AL|                  212247|                  162.9|
|   Huntsville|Alabama|        AL|                  188226|                  157.7|
|       Mobile|Alabama|        AL|                  194675|                  122.5|
|   Montgomery|Alabama|        AL|                  200481|                  129.0|
|Anchorage[19]| Alaska|        AK|                  301010|                   null|
+-------------+-------+----------+------------------------+-----------------------+
only showing top 5 rows



**top 10 most populous cities**

In [0]:
# Order by 2014 Population estimate in descending order and select top 10
top_10_populous_cities = df_selected.orderBy("2014 Population estimate", ascending=False).limit(10)

# Show the results
display(top_10_populous_cities)

City,State,State Code,2014 Population estimate,2015 median sales price
New York[6],New York,NY,8491079,388.6
Los Angeles,California,CA,3928864,434.7
Chicago,Illinois,IL,2722389,192.5
Houston[7],Texas,TX,2239558,200.3
Philadelphia[8],Pennsylvania,PA,1560297,204.9
Phoenix,Arizona,AZ,1537058,206.1
San Antonio,Texas,TX,1436697,184.7
San Diego,California,CA,1381069,510.3
Dallas,Texas,TX,1281047,192.5
San Jose,California,CA,1015785,900.0


Databricks visualization. Run in Databricks to view.

**average median sales price by state**

In [0]:
from pyspark.sql.functions import avg
df_selected.groupBy("state").agg(
    avg("2015 median sales price").alias("avg median price")
).orderBy("avg median price", ascending=False).show(10)

+--------------------+------------------+
|               state|  avg median price|
+--------------------+------------------+
|             Hawai'i|             699.3|
|          California| 547.9714285714286|
|District of Columbia|             367.8|
|          New Jersey|             350.8|
|            Colorado|333.46666666666664|
|          Washington|266.29999999999995|
|       Massachusetts|             261.8|
|         Connecticut|250.13333333333333|
|                Utah|             243.3|
|       New Hampshire|             237.4|
+--------------------+------------------+
only showing top 10 rows



**filter cities with population>1million**

In [0]:
display(df_selected.filter(df_selected["2014 Population estimate"] > 1000000))

City,State,State Code,2014 Population estimate,2015 median sales price
Phoenix,Arizona,AZ,1537058,206.1
Los Angeles,California,CA,3928864,434.7
San Diego,California,CA,1381069,510.3
San Jose,California,CA,1015785,900.0
Chicago,Illinois,IL,2722389,192.5
New York[6],New York,NY,8491079,388.6
Philadelphia[8],Pennsylvania,PA,1560297,204.9
Dallas,Texas,TX,1281047,192.5
Houston[7],Texas,TX,2239558,200.3
San Antonio,Texas,TX,1436697,184.7


**identify cities with missing or zero price values**

In [0]:
df_selected.filter((df_selected["2015 median sales price"].isNull()) | (df_selected["2015 median sales price"] == 0)).show()

+-------------+----------+----------+------------------------+-----------------------+
|         City|     State|State Code|2014 Population estimate|2015 median sales price|
+-------------+----------+----------+------------------------+-----------------------+
|Anchorage[19]|    Alaska|        AK|                  301010|                   null|
|     Chandler|   Arizona|        AZ|                  254276|                   null|
|  Gilbert[20]|   Arizona|        AZ|                  239277|                   null|
|     Glendale|   Arizona|        AZ|                  237517|                   null|
|         Mesa|   Arizona|        AZ|                  464704|                   null|
|       Peoria|   Arizona|        AZ|                  166934|                   null|
|   Scottsdale|   Arizona|        AZ|                  230512|                   null|
|     Surprise|   Arizona|        AZ|                  126275|                   null|
|        Tempe|   Arizona|        AZ|      